In [1]:
import os, sys

import torch

import matplotlib.pyplot as plt

In [2]:
sys.path.append('../..')

In [3]:
from lib.datasets import get_train_test_img_ids_split, get_kfolds
from lib.dataloaders import PatchesDataset
from lib.augmentations import augment_v1

In [4]:
train_img_ids, test_img_ids = get_train_test_img_ids_split()

In [5]:
train_img_ids[:4], test_img_ids[:4]

(['eaf66ac783499827a7f09d69793a07d7',
  'ea035a96c752f59a4934a91068b318f2',
  '4c0d7290c72b5495df6c337b1e5dee88',
  '9edf591d741baf346fc1ff0f42dd4ddc'],
 ['e8baa3bb9dcfb9cef5ca599d62bb8046',
  '9b2948ff81b64677a1a152a1532c1a50',
  '5b003d43ec0ce5979062442486f84cf7',
  '375b2c9501320b35ceb638a3274812aa'])

In [6]:
#%%time
#img, mask, labes, provider, isup_grade, g_score = next(iter(train_loader))

In [7]:
import torch
import torchvision
from torchsummary import summary
from torch import optim

from torch.nn import functional as F
from torch import nn

In [8]:
import pretrainedmodels

In [9]:
# backbone_model = pretrainedmodels.se_resnet50()
# backbone_model = torchvision.models.resnet18(num_classes=9)

In [10]:
#backbone_model.avg_pool = nn.Identity()
#backbone_model.last_linear = nn.Identity()

In [11]:
sys.path.append('../../../captcha/lib/')

In [12]:
from models import resnet_mini_custom

In [13]:
backbone_model = resnet_mini_custom.resnet20_c7_fls2(num_classes=9)

In [14]:
backbone_model.conv1 = nn.Conv2d(3, 16, kernel_size=(7, 7), stride=(2, 2), padding=(1, 1), bias=False)

In [15]:
summary(backbone_model, (3, 512, 512), -1, 'cpu')

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 16, 254, 254]           2,352
       BatchNorm2d-2         [-1, 16, 254, 254]              32
            Conv2d-3           [-1, 16, 64, 64]           2,304
       BatchNorm2d-4           [-1, 16, 64, 64]              32
            Conv2d-5           [-1, 16, 64, 64]           2,304
       BatchNorm2d-6           [-1, 16, 64, 64]              32
       LambdaLayer-7           [-1, 16, 64, 64]               0
        BasicBlock-8           [-1, 16, 64, 64]               0
            Conv2d-9           [-1, 16, 64, 64]           2,304
      BatchNorm2d-10           [-1, 16, 64, 64]              32
           Conv2d-11           [-1, 16, 64, 64]           2,304
      BatchNorm2d-12           [-1, 16, 64, 64]              32
       BasicBlock-13           [-1, 16, 64, 64]               0
           Conv2d-14           [-1, 16,

In [16]:
device = torch.device('cuda:0')

In [17]:
backbone_model.to(device);

In [18]:
from tqdm.auto import tqdm

In [19]:
class SmoothLoss:
    "A simple loss compute and train function."
    def __init__(self, criterion, smoothing=0.1):
        self.criterion = criterion
        self.smoothing = smoothing
        self.confidence = 1 - smoothing
        
    def __call__(self, output, target):
        smooth_target = target
        # smooth_target = torch.full_like(output, self.smoothing / (output.size(1) - 1))
        # smooth_target.scatter_(1, target[:, None, :], self.confidence)
        loss = self.criterion(output, smooth_target)
        return loss

In [20]:
c_criterion0 = SmoothLoss(lambda o, t: F.kl_div(o, t) * o.size(1), smoothing=0.1)
criterion = c_criterion0

In [21]:
model = backbone_model

In [22]:
optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-4)

In [23]:
train_loader = torch.utils.data.DataLoader(
    PatchesDataset(train_img_ids, transform=augment_v1, scale=0.5, load_masks=True),
    batch_size=64, shuffle=True, num_workers=6, pin_memory=True,
)

In [24]:
import itertools

In [25]:
train_loader_iter = iter(itertools.cycle(train_loader))

In [26]:
%%time
img, mask, labels, provider, isup_grade, g_score = next(train_loader_iter)

CPU times: user 443 ms, sys: 119 ms, total: 562 ms
Wall time: 1.15 s


In [27]:
%%time
for b in tqdm(range(120)):
    #model.train()
    
    img, mask, labels, provider, isup_grade, g_score = next(train_loader_iter)
    
    img, labels = img.to(device), labels.to(device)
    
    #optimizer.zero_grad()
    
    #output = model(img)
    
    #loss = criterion(output, labels)
    
    #loss.backward()

    #optimizer.step()


CPU times: user 2.14 s, sys: 3.28 s, total: 5.43 s
Wall time: 18.1 s


In [28]:
# Без сети
# 1 wo aug 12.7 s
# 1 w aug 27.6 s

# 0.5 wo aug 8.57 
# 0.5 w aug 11.3
# 0.5 w aug w masks 18.1


# resnet18
# 0.5 wo aug 15.1
# 0.5 w aug 15.0
# 1 w aug 60.0

# resnet20_c7_fls2
# 1 w aug 00:26 - затык в подготовке данных
# 1 wo aug 00:13